In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

In [22]:
#!cp -r "drive/My Drive/tumor_dataset" 
!cp -r "drive/My Drive/preprocess.py" "preprocess.py"
!cp -r "drive/My Drive/resnet.py" "resnet.py"

In [ ]:
import imutils as imutils
import numpy as np
from tqdm import tqdm
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from preprocess import *
from keras.applications.vgg16 import VGG16, preprocess_input
from resnet import *
import tensorflow as tf
from keras.optimizers import Adam
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import os

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
!mkdir train
!mkdir val
!mkdir test
!mkdir train/yes
!mkdir train/no
!mkdir val/yes
!mkdir val/no
!mkdir test/yes
!mkdir test/no



In [ ]:
IMG_SIZE = (224,224)
X, y, labels = load_data("tumor_dataset/", IMG_SIZE)
cropped_X = crop_imgs(X)
preprocessed_X = preprocess_imgs(cropped_X, IMG_SIZE)
X_train_val, X_test, y_train_val, y_test = train_test_split(preprocessed_X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)
save_new_images(X_train, y_train, folder_name='train/')
save_new_images(X_val, y_val, folder_name='val/')
save_new_images(X_test, y_test, folder_name='test/')
#with open('/content/gdrive/My Drive/TRAIN_CROP/', 'w') as f:
#  f.write(X_train, y_train)
demo_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rescale=1./255,
    shear_range=0.05,
    brightness_range=[0.1, 1.5],
    horizontal_flip=True,
    vertical_flip=True
)


In [ ]:
TRAIN_DIR = 'train/'
VAL_DIR = 'val/'



datagen1 = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.1,
    brightness_range=[-10, 10],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input

)
data = datagen1.flow(X_train, y_train, batch_size=32)
val_data = datagen1.flow(X_val, y_val, batch_size=32)
             
input_shape = X_train.shape[1:]
model = my_resnet(input_shape)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-1), metrics=['accuracy'])

for x, z in data:
    model.train_on_batch(x, z)
for x , z in val_data:
    model.fit(x,z)

a . adding a layer of bath norm helps model to overcome overfitting , we used this layer in out defined model

b . adding a layer of drop out after relu activation helps us to overcome overfitting, there is an option for resnet funtion to have dropout layer but we didnt use it.

